/Users/mariotuci/Desktop/Google-Drive/Master/SoSe-24/Project Studies/Project/Code/latent-communication/optimization
[[ 0.00000000e+00  4.03562401e-01  0.00000000e+00 ...  1.89831485e-01
  -6.14076475e-02  1.34250912e-01]
 [ 0.00000000e+00 -3.00620202e-02  0.00000000e+00 ...  4.04684097e-02
  -6.27929378e-02  1.31249487e-02]
 [ 0.00000000e+00 -1.61046534e-14  0.00000000e+00 ...  1.62092363e-13
  -2.26602676e-13  2.12851149e-14]
 ...
 [ 0.00000000e+00  5.03275279e-01  0.00000000e+00 ...  2.17558779e-01
  -2.42602042e-01  1.68836393e-01]
 [ 0.00000000e+00 -5.93611337e-03  0.00000000e+00 ... -4.92641754e-03
   2.22867583e-03 -3.16713157e-03]
 [ 0.00000000e+00 -3.22006655e-01  0.00000000e+00 ... -6.61202024e-02
   6.77698198e-02 -1.71845469e-01]]


In [ ]:
def pca_def(v):
    pca = PCA(n_components=2)
    pca.fit(v)
    return pca


def plotLatentTransformed(model, A, pca1, name):
    images, _ = next(iter(data_loader.train_loader))
    latent_spaces = []
    all_labels = []
    for images, labels in data_loader.train_loader:
        images = images.to(DEVICE)
        images = images.view(images.size(0), -1)
        latent_space = model.getLatenSpace(images)
        latent_space = latent_space.cpu().detach().numpy()
        latent_spaces.append(latent_space)
        all_labels.append(labels.numpy())

    # Concatenate latent space representations from all batches
    latent_space = np.concatenate(latent_spaces, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    
    # Transform the latent space
    latent_space_transformed = np.dot(latent_space, A.T)
    # Plot latent space via PCA
    latent_space_pca = pca1.transform(latent_space_transformed)

    # Plot the latent space

 
    plot = plt.scatter(latent_space_pca[:, 0], latent_space_pca[:, 1], c=all_labels, cmap='tab10', label=all_labels)
    plt.title('Latent Space '+ name)
    plt.show(plot)


    return  plot

I = np.eye(32)

pca1 = pca_def(z2)
